In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime
import requests, json
from typing import List, Dict
from bs4 import BeautifulSoup as bs

# Stock Prices EDA

## Compute Sharp Ratio

Risk-free rate is ~ $0.08$ for 2021

$s_a=\frac{E[R_a - R_b]}{\sigma_a}$


In [64]:
class SmigStock:

    DATE_FORMAT = "%Y-%m-%d"
    RISK_FREE_RATE = 0.0006

    @property
    def tickers(self):
        return self._tickers

    @tickers.setter
    def tickers(self, tickers: List[str]):
        if tickers and isinstance(tickers, list):
            self._tickers = tickers

    def add_ticker(self, ticker: str) -> None:
        self.tickers.append(ticker)

    @property
    def start_date(self) -> "datetime.datetime":
        return self._start_date

    @start_date.setter
    def start_date(self, value: str) -> None:
        if not isinstance(value, str):
            return
        try:
            self._start_date = datetime.datetime.strptime(value, SmigStock.DATE_FORMAT)
        except Exception as e:
            print(str(e))
            raise

    @property
    def end_date(self) -> "datetime.datetime":
        return self._end_date

    @end_date.setter
    def end_date(self, value: str) -> None:
        if not isinstance(value, str):
            return
        try:
            self._end_date = datetime.datetime.strptime(value, SmigStock.DATE_FORMAT)
        except Exception as e:
            print(str(e))
            raise

    def __init__(self, tickers: List[str], start_date: str, end_date: str) -> None:
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.data = {}

    def _get_trending_stock_twits(self):
        url = 'https://api.stocktwits.com/api/2/streams/trending.json'
        body = requests.get(url).json().get('messages')
        symbols = []
        for x in body:
            for _s in x.get('symbols'):
                symbol = _s.get('symbol')
                if isinstance(symbol, str):
                    self.tickers.append(symbol)

    def _get_trending_stock_screener(self):
        url = 'https://stock-screener.org/trending-stocks.aspx'
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        symbols = []
        for row in soup.find_all('table',{"class":"styled"})[0].tbody.find_all('tr')[1:]:
            symbols.append(row.find_all('td')[0].text)
        return symbols

    def _get_tickers_from_file(self, filename: str = None) -> None:
        fp = filename if filename else 'tickers.txt'
        with open(fp, 'r') as f:
            symbols = f.read().splitlines()
            self.tickers.extend(symbols)

    def _download_data(self, ticker):
        try:
            df = web.DataReader(ticker, 'yahoo', self.start_date, self.end_date)
        except:
            print(f"Error getting data for {ticker}")
            raise
        else:
            df.reset_index(inplace=True)
            df = df[['Date', 'Adj Close']]
            df.columns = ['ds', 'y']
            df.set_index('ds', inplace=True)
            return df

    def get_prices(self) -> None:
        #? Should you redownload price data for all tickers or just ones not already in self.data?
        for ticker in self.tickers:
            if ticker in self.data:
                print(f"{ticker} data has already been downloaded")
                # continue
            try:
                print(f"Downloading stock price data for {ticker}")
                df = self._download_data(ticker)
            except:
                continue
            else:
                self.data[ticker] = {}       
                df = self._compute_metrics(df)
                # print("Download complete")
                self.data[ticker]['prices'] = df
                self.data[ticker]['sharp_ratio'] = self._compute_sharp_ratio(ticker)

    def _compute_metrics(self, df: 'pandas.DataFrame') -> 'pandas.DataFrame':
        df['daily_returns'] = df.diff(axis=0)
        df['daily_percent'] = df['daily_returns'] / df['y'].shift()
        df['excess'] = df.daily_percent - SmigStock.RISK_FREE_RATE
        return df
        
    def _compute_sharp_ratio(self, ticker: str) -> float:
        # grab the data
        df = self.data[ticker]['prices']
        s = (df.shape[0]/np.sqrt(252))*df.daily_percent.mean()/df.daily_percent.std()
        # print("Sharp Ratio for %s = %s" % (ticker, s))
        return s

    def _sort_prices(self, column: str = 'sharp_ratio') -> None:
        """ Sort self._tickers by column """

        # self._tickers is a Dict
        self.data = dict(sorted(self.data.items(), key=lambda item: item[1][column], reverse=True))

    def print_top_tickers(self, n: int = 10, column: str = 'sharp_ratio') -> None:
        topn = dict(sorted(self.data.items(), key=lambda x: x[1][column], reverse=True)[:n+1])
        for key, value in topn.items():
            print("%s | %s | %s" % (key, value[column], value['prices']['y'].values[-1]))

In [65]:
tickers = ['TSLA','AAPL','AMZN']
ss = SmigStock(tickers=tickers,start_date='2020-10-01', end_date='2021-02-12')
ss._get_trending_stock_twits()
ss._get_trending_stock_screener()
ss._get_tickers_from_file()
ss.get_prices()
ss._sort_prices()

SOS data has already been downloaded
SOS data has already been downloaded
SOS data has already been downloaded
SOS data has already been downloaded
CCIV data has already been downloaded
Error getting data for BTC.X
EBON data has already been downloaded
SOS data has already been downloaded
CCIV data has already been downloaded
SOS data has already been downloaded
EH data has already been downloaded
SOS data has already been downloaded
CCIV data has already been downloaded
SOS data has already been downloaded
SOS data has already been downloaded
EH data has already been downloaded
KBNT data has already been downloaded
EH data has already been downloaded
SOS data has already been downloaded
CCIV data has already been downloaded
SOS data has already been downloaded
SOS data has already been downloaded
AMZN data has already been downloaded
AAPL data has already been downloaded
TSLA data has already been downloaded
Error getting data for BJR


In [66]:
ss.print_top_tickers(n=20)

TPR | 2.197022072720349 | 38.310001373291016
CPRI | 2.052411158193114 | 46.0099983215332
RIOT | 1.9557552610798676 | 49.279998779296875
SIVB | 1.9436292822213066 | 506.5
EH | 1.9230987292271131 | 124.08999633789062
MARA | 1.8393380473305243 | 38.459999084472656
IAC | 1.8307619379394673 | 262.3500061035156
IIVI | 1.7135804513381776 | 89.91999816894531
ON | 1.6823451551803899 | 41.27000045776367
UAA | 1.6691982663778002 | 22.3700008392334
AMAT | 1.6619993310559407 | 116.69999694824219
GS | 1.5230171314618874 | 306.32000732421875
CCIV | 1.520608594983176 | 39.97999954223633
VRNS | 1.4615276083555657 | 214.5
LRCX | 1.4308138879245658 | 586.1599731445312
BOOT | 1.3976516915751722 | 60.880001068115234
WDC | 1.3427630197190217 | 65.56999969482422
ENPH | 1.3423084159417862 | 206.50999450683594
EGHT | 1.2860101385565637 | 35.900001525878906
SNAP | 1.2608226881368236 | 61.970001220703125
STX | 1.2382389237809213 | 72.20999908447266
